In [ ]:
!pip uninstall keras
!pip install keras==2.3.1

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.1.0

In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
!pip install matchzoo

In [ ]:
### INIT ###

import keras
import pandas as pd
import numpy as np
import matchzoo as mz
import json

print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task='ranking')
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

print("loading embedding ...")
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
print("embedding loaded as `glove_embedding`")

In [ ]:
def append_params_to_readme(model):
    import tabulate
    
    with open('README.rst', 'a+') as f:
        subtitle = model.params['model_class'].__name__
        line = '#' * len(subtitle)
        subtitle = subtitle + '\n' + line + '\n\n'
        f.write(subtitle)
        
        df = model.params.to_frame()[['Name', 'Value']]
        table = tabulate.tabulate(df, tablefmt='rst', headers='keys') + '\n\n'
        f.write(table)

  ### END INIT ### 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
### KNRM ###

preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

In [ ]:
preprocessor.context

In [ ]:
model = mz.models.KNRM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()

In [ ]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [ ]:
pred_x, pred_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x), model_save_path='./knrm_pretrained_model/')

In [ ]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

In [ ]:
history = model.fit_generator(train_generator, epochs=15, callbacks=[evaluate], workers=30, use_multiprocessing=True)

In [ ]:
model.evaluate(pred_x, pred_y)
model.predict(pred_x)